In [1]:
import xlwings as xw
from xlwings.constants import Direction
import pandas as pd
import datetime as dt
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

class HistoryData:
    def open_file(self, filename, sheetname):
        self.wb = xw.Book(filename)
        self.sheet = self.wb.sheets[sheetname]

        # 找最後一筆資料的row數值
        last_cell = self.sheet.cells(1, "A").end(Direction.xlDown)
        self.last = last_cell.row
    
    
    def close_file(self):
        self.wb.save()
        self.wb.close()
    
    
    def find_date(self):
        while True:
            try:
                # 設立頭尾範圍
                start_date = input('請輸入想要查找的開始日期(例:2023/1/1)，輸入"q"或"Q"返回首頁:')
                if start_date == 'q' or start_date == 'Q':
                    print('='*30)
                    break
                s_y, s_m, s_d = start_date.split('/')
                self.start = dt.datetime(int(s_y), int(s_m), int(s_d))
                start_date = dt.datetime(int(s_y), int(s_m), int(s_d))
                end_date = input('請輸入想要查找的結束日期(例:2023/12/31)，輸入"q"或"Q"返回首頁:')
                if end_date == 'q' or end_date == 'Q':
                    print('='*30)
                    break
                e_y, e_m, e_d = end_date.split('/')
                self.end = dt.datetime(int(e_y), int(e_m), int(e_d))
                end_date = dt.datetime(int(e_y), int(e_m), int(e_d))

                if start_date > end_date:
                    print('請按照「開始日期」與「結束日期」順序輸入')
                    print('-'*30)
                    continue

                # 在csv中尋找結束、開始位置
                for i in range(2, self.last+1):
                    if self.sheet.cells(i, 'A').value <= end_date:
                        self.end = self.sheet.cells(i, 'A').row
                        break
                for i in range(2, self.last+1):
                    if self.sheet.cells(i, 'A').value == start_date:
                        self.start = self.sheet.cells(i, 'A').row
                        break
                    elif self.sheet.cells(i, 'A').value < start_date:
                        self.start = self.sheet.cells(i-1, 'A').row
                        break
            except:
                print('請輸入正確的日期格式')
                print('-'*30)
                continue
            break

        
    def df_build(self):
        # 建立DataFrame
        self.hist_data = pd.DataFrame({
            self.sheet.cells(1, 'A').value: [],
            self.sheet.cells(1, 'B').value: [],
            self.sheet.cells(1, 'C').value: [],
            self.sheet.cells(1, 'D').value: [],
            self.sheet.cells(1, 'E').value: [],
            self.sheet.cells(1, 'F').value: [],
            self.sheet.cells(1, 'G').value: [],
            self.sheet.cells(1, 'H').value: [],
        })
        
        # 結束位置、開始位置寫進df
        li = self.sheet.range('A'+f'{self.end}'+':A'+f'{self.start}').value
        self.hist_data['日期'] = li
        li = self.sheet.range('B'+f'{self.end}'+':B'+f'{self.start}').value
        self.hist_data['開盤'] = li
        li = self.sheet.range('C'+f'{self.end}'+':C'+f'{self.start}').value
        self.hist_data['最高'] = li
        li = self.sheet.range('D'+f'{self.end}'+':D'+f'{self.start}').value
        self.hist_data['最低'] = li
        li = self.sheet.range('E'+f'{self.end}'+':E'+f'{self.start}').value
        self.hist_data['收盤'] = li
        li = self.sheet.range('F'+f'{self.end}'+':F'+f'{self.start}').value
        self.hist_data['漲跌'] = li
        li = self.sheet.range('G'+f'{self.end}'+':G'+f'{self.start}').value
        self.hist_data['漲跌(%)'] = li
        li = self.sheet.range('H'+f'{self.end}'+':H'+f'{self.start}').value
        self.hist_data['成交張數'] = li
    
    
    def df_display(self):
        # 對齊中英文格式
        display(self.hist_data)
        
        
    def show_price_volume_chart(self):
        # 設定中文字體
        plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
        
        # 開新畫布、設立標題
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(111) # 等於add_subplot(1, 1, 1)
        plt.title('股票價量走勢圖')
        
        # 設定直方圖x,y軸參數
        plt.ylabel('成交量(張)')
        height = [i for i in range(len(self.hist_data))]
        date = [str(i.year)+'/'+str(i.month)+'/'+str(i.day) for i in self.hist_data['日期']]
        labels = date
        
        # 畫柱狀圖
        plt.bar(height, self.hist_data['成交張數'], color='pink', label='交易量(張)')
        plt.xticks(height, labels, rotation='vertical')
        plt.legend(loc='upper left')
        
        # 設定折線圖參數
        ax2 = plt.twinx()
        ax2.set_ylabel('股價')
        
        # 畫折線圖
        plt.plot(labels, self.hist_data['收盤'], color='navy', marker='.', markersize='8', label='股價')
        
        # 顯示折線圖數值
        for a, b in zip(labels, self.hist_data['收盤']):
            plt.text(a, b, b, ha='center', va='bottom', fontsize='6')
        
        # 圖調整
        tick_spacing = int(len(self.hist_data['日期']) / 10)
        ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
        plt.xlabel('日期')
        plt.grid(True, axis='y')
        plt.legend(loc='upper right')
        
        plt.show()
        
        
    def analyze_per(self, per, title, cur_price):
        # 本週預估EPS
        cur_PER = per['目前PER(倍)']
        cur_PER = cur_PER.iloc[0]
        if cur_PER:
            cur_PER = round(float(cur_PER))

            # 計算前一年平均EPS,取小數點後兩位
            avg_EPS = []
            EPS = per['河流圖EPS(元)']
            week = per['交易週別']
            count = 0
            last_year = dt.date.today().year % 100 - 1
            for w in week:
                if int(w[:2]) == last_year:
                    avg_EPS.append(EPS.iloc[count])
                count += 1
            avg_EPS = [float(i) for i in avg_EPS]
            avg_EPS = sum(avg_EPS) / len(avg_EPS)
            avg_EPS = round(avg_EPS, 2)

            # 計算合理、較貴的股價(以前一年的每股盈餘為基準)
            last_PER = round(float(cur_price) / avg_EPS)
            should_be_PER = round((cur_PER + last_PER) / 2)
            should_be_price = round(should_be_PER * float(EPS[0]), 1)

            # 分析報告
            print('*****此分析是針對去年整年的價格比較，不建議短期或當沖用戶參考*****')
            print('-'*60)
            print('目前股價為:', cur_price, '，股價的本益比為:', cur_PER, '倍')
            print('目前的每股盈餘為:', EPS[0], '，去年的每股盈餘平均為:', avg_EPS, '。與去年平均相比:', round(float(EPS[0]) - avg_EPS, 2))

            # 計算股票狀況
            print('依照去年每股盈餘平均('+str(avg_EPS)+')的標準，股票合理本益比為:', last_PER, '倍')
            if (cur_PER >= last_PER) and (cur_PER > 20):
                print('該支股票目前股價「遠高於」去年標準，且本益比大於市場交易價格')
                print('-'*30)
                print('可能情況1: 該股票正處於成長期且被投資人看好，進而高出每股盈餘的20倍以上')
                print('可能情況2: 該股票不利於長期持有，可能會泡沫化')
                print('建議: 可以多觀察市場情況，合理的價格約落在', should_be_price, '，注意不要急著買進以免買貴')
            elif (cur_PER >= last_PER) and (12 <= cur_PER <= 20):
                print('該支股票目前股價高於去年標準，但還算落在市場合理的交易價格內')
                print('-'*30)
                print('可能情況: 該公司目前處於合理價格的本益比12~20倍，並被投資人看好')
                print('建議: 可以多觀察市場情況，便宜的價格約落在', should_be_price)
            elif (cur_PER >= last_PER) and (cur_PER < 12):
                print('該支股票目前低於去年標準，但相較去年，股價仍有成長')
                print('-'*30)
                print('可能情況1: 可以再多觀望一陣子，看股票走勢是否有機會成長')
                print('可能情況2: 該公司目前可能位於成長前期，屬於高價值投資標的')
                print('建議: 可以多觀察市場情況，合理的進場價格約落在', should_be_price)

            elif (cur_PER < last_PER) and (cur_PER > 20):
                print('該支股票目前股價低於去年標準')
                print('-'*30)
                print('可能情況1: 股價稍微動盪，可能只是一時的價格波動')
                print('可能情況2: 考慮到目前股價「遠高於」市場標準，股價可能面臨暴風雨前的寧靜，要注意出手時間')
                print('建議: 可以多觀察市場情況，合理的價格約落在', should_be_price)
            elif (cur_PER < last_PER) and (12 <= cur_PER <= 20):
                print('該支股票目前股價低於去年標準，但還算落在市場合理的交易價格內')
                print('-'*30)
                print('可能情況1: 目前投資人不怎麼看好該支股票，可能有繼續下跌的趨勢')
                print('可能情況2: 目前投資人不怎麼看好該支股票，但考慮到還在合理價格的本益比12~20倍範圍，是一個不錯的進場時機')
                print('建議: 可以多觀察市場情況，便宜的價格約落在', should_be_price)
            elif (cur_PER < last_PER) and (cur_PER < 12):
                print('該支股票目前股價「遠低於」去年標準，且本益比小於市場價格')
                print('-'*30)
                print('可能情況1: 該公司處於不優良的價格，繼續持有可能會有套牢的風險')
                print('可能情況2: 該公司目前價格很接近市場價格，只是不怎麼吸引投資人')
                print('可能情況3: 該公司目前可能位於成長前期，屬於高價值投資標的')
                print('建議: 可以多觀察市場情況，適合的進場價格約落在', should_be_price)
        
        